In [1]:
from hdf5storage import loadmat, savemat 
import numpy as np 

from scipy import signal 
from scipy.fftpack import fft

from sklearn.covariance import GraphicalLassoCV

import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__)) 
# make sure it is newer version, works in version 1.2.1

The scikit-learn version is 0.21.3.


/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/utils/multiclass.py:13: DeprecationWarning: Please use `spmatrix` from the `scipy.sparse` namespace, the `scipy.sparse.base` namespace is deprecated.
  from scipy.sparse.base import spmatrix
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/zhibinz2/anaconda3/envs/eeg/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, 

In [2]:
# load abs_zscore_all
outdict=loadmat('../Pcorr_single_model/abs_zscore_all.mat')
abs_zscore_all=outdict["abs_zscore_all"]

In [3]:
# append abs_zscore from 24 subject together
abs_zscore_subj_L_append =  [[] for i in range(12)]
for ses in range(12):
    for trl in range(12):
        abs_zscore_subj_L_append[ses].append(abs_zscore_all[ses][trl][0]) 

abs_zscore_subj_R_append =  [[] for i in range(12)]
for ses in range(12):
    for trl in range(12):
        abs_zscore_subj_R_append[ses].append(abs_zscore_all[ses][trl][1]) 

abs_zscore_subj_append=[abs_zscore_subj_L_append, abs_zscore_subj_R_append]


In [4]:
ampdata_subj_L =  [[] for i in range(12)]
ampdata_subj_R =  [[] for i in range(12)]
ampdata_subj   =  [ampdata_subj_L, ampdata_subj_R]

In [5]:
# concatenate the trials in each subject
for subj in range(2):
    for ses in range(12):
        ampdata_subj[subj][ses]=np.concatenate(abs_zscore_subj_append[subj][ses])